### Efficient Yelp API Calls (Core)
#### Sarah Musselwhite
#### 10/19/2022

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
with open('/Users/Sarah/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()


dict_keys(['client-id', 'api-key'])

In [3]:
# Create an instance with your key
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)
yelp_api

In [4]:
# set our API call parameters 
LOCATION = 'Nashville, TN'
TERM = 'Italian'

In [5]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_NashTN_Italian.json"
JSON_FILE

'Data/results_in_progress_NashTN_Italian.json'

In [6]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_NashTN_Italian.json not found. Saving empty list to file.


In [7]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [8]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [9]:
## How many results total?
total_results = results['total']
total_results

573

In [10]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [11]:
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

29

In [12]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [13]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    # add a 200ms pause
    time.sleep(.2)

  0%|          | 0/29 [00:00<?, ?it/s]

In [14]:
os.remove(JSON_FILE)
os.path.isfile(JSON_FILE)

False

In [15]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [16]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_NashTN_Italian.json not found. Saving empty list to new file.
- 0 previous results found.


29

In [17]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/29 [00:00<?, ?it/s]

In [18]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,QL4C7sd9PFUOGGpHwe1dlQ,trattoria-il-mulino-nashville,Trattoria Il Mulino,https://s3-media1.fl.yelpcdn.com/bphoto/T9cubd...,False,https://www.yelp.com/biz/trattoria-il-mulino-n...,369,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 36.15976916947817, 'longitude': -...",[delivery],$$$,"{'address1': '144 Rep John Lewis Way S', 'addr...",+16156203700,(615) 620-3700,2463.163444
1,rCOIoZ3WuA6pkgRltfEN1g,amerigo-italian-restaurant-nashville-4,Amerigo Italian Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/5DE7_0...,False,https://www.yelp.com/biz/amerigo-italian-resta...,587,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",4.0,"{'latitude': 36.1516729421203, 'longitude': -8...","[delivery, pickup]",$$,"{'address1': '1920 West End Ave', 'address2': ...",+16153201740,(615) 320-1740,2640.168504
2,vVAOIMU5cz8PL4d8Dzeb5Q,tutti-da-gio-hermitage,Tutti Da Gio,https://s3-media2.fl.yelpcdn.com/bphoto/UMhWAg...,False,https://www.yelp.com/biz/tutti-da-gio-hermitag...,50,"[{'alias': 'sicilian', 'title': 'Sicilian'}, {...",5.0,"{'latitude': 36.16748676542455, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '5851 Old Hickory Blvd', 'address...",+16156226078,(615) 622-6078,16040.831753
3,LxsWPSt_Su65dB48iH-bkA,il-forno-nashville-2,Il Forno,https://s3-media3.fl.yelpcdn.com/bphoto/XnrUVP...,False,https://www.yelp.com/biz/il-forno-nashville-2?...,82,"[{'alias': 'italian', 'title': 'Italian'}]",4.0,"{'latitude': 36.1413, 'longitude': -86.7627}","[delivery, pickup]",$$,"{'address1': '1414 3rd Ave S', 'address2': 'St...",+16157396713,(615) 739-6713,1216.351274
4,IkLp51a_x_0CPlg0qMEqNg,pennepazze-nashville,PennePazze,https://s3-media2.fl.yelpcdn.com/bphoto/tvj-pl...,False,https://www.yelp.com/biz/pennepazze-nashville?...,232,"[{'alias': 'italian', 'title': 'Italian'}]",4.5,"{'latitude': 36.1526515, 'longitude': -86.8327...","[delivery, pickup]",$$,"{'address1': '3826 Charlotte Ave', 'address2':...",+16157303088,(615) 730-3088,5418.091175


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
568,r0iqLPjPiYVHC8C-Hvy7mQ,dominos-pizza-franklin-8,Domino's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/22qrnm...,False,https://www.yelp.com/biz/dominos-pizza-frankli...,29,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.5,"{'latitude': 35.956809, 'longitude': -86.802401}",[],$,"{'address1': '9040 Carothers Pkwy', 'address2'...",+16154358181,(615) 435-8181,20258.700925
569,E7bkcP9nsg448iz3NLtRXA,jets-pizza-murfreesboro-2,Jet's Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/EiP0uQ...,False,https://www.yelp.com/biz/jets-pizza-murfreesbo...,20,"[{'alias': 'pizza', 'title': 'Pizza'}]",2.0,"{'latitude': 35.9073486, 'longitude': -86.3842...","[delivery, pickup]",$$,"{'address1': '3325 Memorial Blvd', 'address2':...",+16158965550,(615) 896-5550,43522.573229
570,t-2usJ_mJs6wAXIlBwGZFA,dominos-pizza-murfreesboro,Domino's Pizza,https://s3-media1.fl.yelpcdn.com/bphoto/yYzJfE...,False,https://www.yelp.com/biz/dominos-pizza-murfree...,20,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.0,"{'latitude': 35.8247414, 'longitude': -86.41745}",[],$,"{'address1': '1515 Warrior Dr', 'address2': ''...",+16158900893,(615) 890-0893,47424.242961
571,QHRaB7S6I0HET0uLgR3FyA,papa-johns-pizza-murfreesboro-3,Papa Johns Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/wHUtrg...,False,https://www.yelp.com/biz/papa-johns-pizza-murf...,22,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.0,"{'latitude': 35.88736084483143, 'longitude': -...","[delivery, pickup]",$,"{'address1': '2365 Memorial Blvd', 'address2':...",+16158907272,(615) 890-7272,44879.967486
572,zZbCXMlme_rnBc7Khzyodg,pizza-hut-murfreesboro,Pizza Hut,https://s3-media1.fl.yelpcdn.com/bphoto/8s7Ffc...,False,https://www.yelp.com/biz/pizza-hut-murfreesbor...,18,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.0,"{'latitude': 35.844851, 'longitude': -86.443754}","[delivery, pickup]",$,"{'address1': '2575 Old Ford Pkwy', 'address2':...",+16152177557,(615) 217-7557,44172.862064


In [19]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

1

In [20]:
## Drop duplicate ids and confirm there are no more duplicates
final_df = final_df.drop_duplicates(subset='id')
final_df.duplicated(subset='id').sum()

0

In [21]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_NashTN_Italian.csv.gz', compression='gzip',index=False)